### OpenWeatherMap
http://openweathermap.org/      
기본적으로 유료이지만, 현재 날씨와 5일까지의 날씨는 무료로 사용가능
- json data

In [7]:
import requests
import json

In [51]:
apikey = "d5265640d43cf3f4ade53461a389e5fb"
# 날씨를 확인할 도시 지정하기
cities = ['Seoul,KR','Tokyo,JP','New York,US','Beijing,CN']


In [52]:
api = 'http://openweathermap.org/data/2.5/weather?1={city}&APPID={key}'

In [53]:
# 켈빈 온도를 섭씨 온도로 변환
k2c = lambda k: k -273.15

In [55]:
url = api.format(city=cities[0],key=apikey)
r = requests.get(url)
data = json.loads(r.text)
# print(data)
print(" + 도시 = ", data['name'])
print(" | 날씨 = ", data['weather'][0]['description'])
print(" | 최저 기온 = ", k2c(data['main']['temp_min']))

KeyError: 'name'

---
### Naver API로 위도 경도 가져오기

Client ID: 8ql50uk6tx   
Client Secret: X8wts2TLSDNGq843fmQEMw6uifYRhICmxlrDg0Fz

In [8]:
import urllib.request
import urllib

In [ ]:
client_id = "8ql50uk6tx"
client_secret = "X8wts2TLSDNGq843fmQEMw6uifYRhICmxlrDg0Fz"

---
### 주소 위도 경도 찾기
- urllib.parse.quote("여기에 주소(지번)을 입력하면")
- 아래 process를 거쳐 위도 경도가 출력됨.

In [59]:
encText = urllib.parse.quote("서울시 영등포구 국회대로 608")
url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=" + encText # json 결과
request = urllib.request.Request(url)
request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
request.add_header("X-NCP-APIGW-API-KEY",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    entext = response_body.decode('utf-8')
else:
    print("Error Code:" + rescode)

In [ ]:
import json    
data1 = json.loads(entext)
# print(data1)
data1

In [63]:
# 경도
lng = data1['addresses'][0]['x']
# 위도
lat = data1['addresses'][0]['y']

print(lng,lat)

126.9009426 37.5258476


# 주소 입력하면 경도 위도 반환하는 process

In [9]:
import pandas as pd

In [10]:
# 저장한 가장맛있는족발 지점 데이터 csv파일을 불러오기
df = pd.read_csv("../Data/Gajok.csv")
df.reset_index(drop=True)
address_list = [df.Address]

name_list = [df.Branch]



In [ ]:
# 가족 지점 주소 입력

client_id = "8ql50uk6tx"
client_secret = "X8wts2TLSDNGq843fmQEMw6uifYRhICmxlrDg0Fz"
name_data = []
address_data = []
xy_data = []

for i in range(len(df)):
    name = name_list[0][i]
    address = address_list[0][i]
    try:
        encText = urllib.parse.quote(address)
        url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=" + encText # json 결과
        request = urllib.request.Request(url)
        request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
        request.add_header("X-NCP-APIGW-API-KEY",client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            entext = response_body.decode('utf-8')
            
            data = json.loads(entext)
            # 경도
            lng = data['addresses'][0]['x']
            # 위도
            lat = data['addresses'][0]['y']
            # 경도 위도 한 컬럼으로 join
            xy_join = [lng,lat]
            xy = ', '.join(xy_join)

            # 리스트에 추가
            name_data.append(name)
            address_data.append(address)
            xy_data.append(xy)

            # 지점명
            print('------------------------------')
            print('지점명: ',name,'\n지점주소:\t',address,'\n위도/경도: ',lng,lat)

        else:
            print("Error Code:" + rescode)
    except:
        print('---------------Error---------------')
        print(name)
        print(address)
        print('---------------Error---------------')
        pass

gajok2 = pd.DataFrame(data=zip(name_data,address_data,xy_data), columns=["Branch","Address","Location"], index=None)


In [27]:
gajok2.to_csv("../Data/Gajok2.csv",encoding="utf-8",index=False)

In [28]:
pd.read_csv("../Data/Gajok2.csv")

,Branch,Address,Location
0,강남1호점,서울시 서초구 서초대로73길 42(서초동),"127.0244570, 37.5012293"
1,선정릉점,서울특별시 강남구 봉은사로57길 5(삼성동),"127.0451360, 37.5112143"
2,한남오거리점,서울시 용산구 한남대로 21길 32(한남동),"127.0063017, 37.5334689"
3,일원점,서울시 강남구 일원로 9길 55(일원동),"127.0859860, 37.4907614"
4,뚝섬역점,서울시 성동구 상원1길 42(성수동),"127.0471344, 37.5470048"
...,...,...,...
385,거제아주동점,경남 거제시 아주동 1688-10,"128.6886626, 34.8665622"
386,거제장승포점,경남 거제시 능포동 624-2,"128.7319244, 34.8773676"
387,제주삼화점,제주특별자치도 제주시 화북일동 1023-4,"126.5757279, 33.5144570"
388,제주이도점,제주특별자치도 제주시 승천로72(아라이동),"126.5424650, 33.4923034"


In [37]:
# 현재 df의 컬럼은 Branch, Address, Tel
# gajok2의 컬럼은 Branch, Address, Location

print(df.shape)
# df: 403rows, 3cols

print(gajok2.shape)
# gajok2: 390rows, 3cols

# merge를 통해 공통된 컬럼 (Branch, Address)를 기준으로 Dataframe 병합
gajok3 = pd.merge(df,gajok2,on=("Branch","Address"))
print(gajok3.shape)
# gajok3 결과: 390rows, 4cols
gajok3.to_csv("../Data/Gajok3.csv",encoding="utf-8",index=False)

(403, 3)
(390, 3)
(390, 4)


In [38]:
pd.read_csv("../Data/Gajok3.csv")

,Branch,Address,Tel,Location
0,강남1호점,서울시 서초구 서초대로73길 42(서초동),02-595-5849,"127.0244570, 37.5012293"
1,선정릉점,서울특별시 강남구 봉은사로57길 5(삼성동),02-543-5849,"127.0451360, 37.5112143"
2,한남오거리점,서울시 용산구 한남대로 21길 32(한남동),02-797-1231,"127.0063017, 37.5334689"
3,일원점,서울시 강남구 일원로 9길 55(일원동),02-3412-0070,"127.0859860, 37.4907614"
4,뚝섬역점,서울시 성동구 상원1길 42(성수동),02-468-7535,"127.0471344, 37.5470048"
...,...,...,...,...
385,거제아주동점,경남 거제시 아주동 1688-10,055-681-8682,"128.6886626, 34.8665622"
386,거제장승포점,경남 거제시 능포동 624-2,055-681-5853,"128.7319244, 34.8773676"
387,제주삼화점,제주특별자치도 제주시 화북일동 1023-4,064-722-5849,"126.5757279, 33.5144570"
388,제주이도점,제주특별자치도 제주시 승천로72(아라이동),064-723-5849,"126.5424650, 33.4923034"
